<a href="https://colab.research.google.com/github/saisampathkumar/AI-Cybersecurity/blob/master/ICP8/L5_Encrypted_Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

### Step 1: Create Data Owners

In [1]:
!pip install syft

     |████████████████████████████████| 225kB 4.8MB/s 
     |████████████████████████████████| 389kB 42.8MB/s 
     |████████████████████████████████| 256kB 46.1MB/s 
     |████████████████████████████████| 450kB 39.2MB/s 
     |████████████████████████████████| 204kB 43.8MB/s 
     |████████████████████████████████| 81kB 17.7MB/s 
     |████████████████████████████████| 1.4MB 39.5MB/s 
     |████████████████████████████████| 51kB 21.4MB/s 
     |████████████████████████████████| 276kB 42.4MB/s 
     |████████████████████████████████| 122kB 45.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/ad/9a/f4/3105b5209674ac77fcca7fede95184c62a95df0196888e0e76
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: msgpack 0.5.6
    Uninstalling msgpack-0.5.6:
      Successfully uninstalled msgpack-0.5.6
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
   

In [2]:
import syft as sy
import torch as th
from torch import nn, optim

hook = sy.TorchHook(th)

W0711 23:46:59.985495 140657581631360 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0711 23:47:00.006022 140657581631360 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [3]:
# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

# this step is important in the real-world application.
# You need to inform the workers of others existance
# you will probably have an ssh or http worker, not a virtual worker.

bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

W0711 23:47:04.532591 140657581631360 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0711 23:47:04.534571 140657581631360 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0711 23:47:04.537063 140657581631360 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0711 23:47:04.538228 140657581631360 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0711 23:47:04.540493 140657581631360 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0711 23:47:04.542607 140657581631360 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker #objects:0>

In [0]:
# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)


### Step 2: Create the Model

In [0]:
# Iniitalize A Toy Model
model = nn.Linear(2,1)

### Step 3: Send a Copy of The Model to Alice and Bob

In [0]:
# Iniitalize A Toy Model
model = nn.Linear(2,1)
bobs_model = model.copy().send(bob) 
alices_model = model.copy().send(alice) 

bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)


### Step 4: Train Bob's and Alice's Models (in parallel)

In [7]:
for i in range(10):

    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).mean()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data
    print(f'Bob loss {bobs_loss}')

    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).mean()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    print(f'alice loss {alices_loss}')


Bob loss 0.29452383518218994
alice loss 3.5850563049316406
Bob loss 0.16278186440467834
alice loss 1.0812102556228638
Bob loss 0.09081988036632538
alice loss 0.3388651907444
Bob loss 0.051446881145238876
alice loss 0.11766763776540756
Bob loss 0.029844610020518303
alice loss 0.05074869096279144
Bob loss 0.017937205731868744
alice loss 0.02958858013153076
Bob loss 0.01132302638143301
alice loss 0.02207973785698414
Bob loss 0.0076027014292776585
alice loss 0.01871573179960251
Bob loss 0.0054680341854691505
alice loss 0.0166756771504879
Bob loss 0.004205487668514252
alice loss 0.015116829425096512


### Step 5: Send Both Updated Models to a Secure Worker

In [0]:
alices_model.move(secure_worker) # the move function iterates on all objects in alice and call send on it. NOTEICE: inline function
bobs_model.move(secure_worker)


In [9]:
secure_worker._objects # can you identify those objects in the Secure worker?

{5294136445: Parameter containing:
 tensor([-0.0824], requires_grad=True), 19563256818: Parameter containing:
 tensor([0.3626], requires_grad=True), 49859630107: Parameter containing:
 tensor([[-0.3522,  0.0904]], requires_grad=True), 58189586509: Parameter containing:
 tensor([[0.5030, 0.2318]], requires_grad=True)}

### Step 6: Average The Models

In [10]:
# A pytorch trick: to set the weights: you have to either access the data[idx] 
# or wrap the set function with (with torch.no_grad():)
# below we illustrate the first way
# the next cell demonstrates the complete way:

model.weight.data[0].set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
model.bias.data[0].set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())

tensor([0.1401])

### Step 7: The complete solution over multiple training sessions

In [11]:
iterations = 10
worker_iters = 5

for a_iter in range(iterations):

    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

    for wi in range(worker_iters):
        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target) ** 2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target) ** 2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data

    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)

    with th.no_grad():

        model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
        model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.0079) Alice:tensor(0.0324)
Bob:tensor(0.0011) Alice:tensor(0.0264)
Bob:tensor(0.0005) Alice:tensor(0.0134)
Bob:tensor(0.0011) Alice:tensor(0.0064)
Bob:tensor(0.0017) Alice:tensor(0.0031)
Bob:tensor(0.0020) Alice:tensor(0.0015)
Bob:tensor(0.0021) Alice:tensor(0.0008)
Bob:tensor(0.0019) Alice:tensor(0.0004)
Bob:tensor(0.0017) Alice:tensor(0.0002)
Bob:tensor(0.0014) Alice:tensor(0.0001)


In [0]:
preds = model(data)
loss = ((preds - target) ** 2).mean()

In [13]:
print(preds)
print(target)
print(loss.data)

tensor([[0.1155],
        [0.1003],
        [0.8721],
        [0.8569]], grad_fn=<AddmmBackward>)
tensor([[0.],
        [0.],
        [1.],
        [1.]], requires_grad=True)
tensor(0.0151)


# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [0]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [15]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [16]:
bob_x_share *= 2
alice_x_share *= 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

This even works for addition between two shared values!!

In [17]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted ( you are the dealer, thus still can see --and know everything-- but Alice and Bob did not really know anything about the overall deal --the sum of the numbers; the secrete--)

In this case, however, to keep the secrete is not about the relaiability / integrity of the person who holds it, rather the fact that those shareholders do not know each other!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant.

In [18]:
x = 5

Q = 9923 # the little fix; a very large prime number

bob_x_share = 1525 # give bob a random number.. try with negatives

alice_x_share = Q - bob_x_share + x

print(f'Bob share= {bob_x_share}')
print(f'Alice share= {alice_x_share}')
print(f'Add both shares= {bob_x_share + alice_x_share}... Wrong secrete')

Bob share= 1525
Alice share= 8403
Add both shares= 9928... Wrong secrete


In [19]:
# Decryption happens by summing all the shares together MODULUS some constant

(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so:  `x_share = (2,5,7)`

Even though normally those shares would be distributed among several workers, you can store them in ordered tuples like this for now.

In [0]:
import random

Q = 23740629843760239486723

# accepts a secrete and shares it among n_shahre, returns a tuple with n_share shares
def encrypt(x, n_share=3):
    
    shares = list()
    
    for i in range(n_share-1):
        shares.append(random.randint(0,Q))
        
    shares.append(Q - (sum(shares) % Q) + x)
    
    return tuple(shares)

  # accepts a tuple of shares and return the decrypted values
def decrypt(shares):
    return sum(shares) % Q

In [21]:
shares = encrypt(x=7, n_share=10)
shares

(11170944676616813875743,
 19652317086921857422392,
 7083949659751680580008,
 14386561511855862133328,
 7130618525340826972756,
 16529298872858924093525,
 15163680670774749434948,
 7124955104580047014032,
 10689499862326773630414,
 9771323247773662276476)

In [22]:
decrypt(shares)

7

In [0]:
def add(a, b):
    c = list()
    assert(len(a) == len(b))
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)

In [24]:
# run this block of code multiple times, what do you notice baout x and y?

x = encrypt(5)
print(x)
y = encrypt(7)
print(y)

z = add(x,y)
decrypt(z)

(10972137954447010063498, 5552813780482901845002, 7215678108830327578228)
(5284744256083137879534, 11650446762849984481798, 6805438824827117125398)


12

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [0]:
BASE= 10  # integer numbers (normal numbers)
PRECISION= 3  # how many porecisions after the decimal point to encode
Q = 23740629843760239486723 

In [0]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [27]:
encode(-0.600)

23740629843760239345664

In [28]:
decode(9323)

9.323

In [29]:
# run this cell multiple times and notice the changes

x = encrypt(encode(5.5))
y = encrypt(encode(5.5))
z = add(x,y)


print(f'Value of x = {x}')
print(f'Value of y = {y}')
print(f'Value of sum = {z}')
print(f'decoded sum = {decode(decrypt(z))}')

Value of x = (8059099690641802133164, 12535877391400288942701, 3145652761718148416358)
Value of y = (639067934742730256117, 19462377913739931133931, 3639183995277578102175)
Value of sum = (8698167625384532389281, 8257625461379980589909, 6784836756995726518533)
decoded sum = 11.0


# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [30]:
# Its a good idea to restrat the environment at this point

import syft as sy
import torch as th
from torch import nn, optim

hook = sy.TorchHook(th)

# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")


bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

W0711 23:47:05.970136 140657581631360 hook.py:98] Torch was already hooked... skipping hooking process
W0711 23:47:05.972906 140657581631360 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0711 23:47:05.974457 140657581631360 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0711 23:47:05.976083 140657581631360 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0711 23:47:05.977975 140657581631360 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0711 23:47:05.979420 140657581631360 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0711 23:47:05.980396 140657581631360 base.py:628] Worker bob already exists. Replacing 

<VirtualWorker id:secure_worker #objects:44>

In [0]:
x = th.tensor([1,2,3,4,5]) ##### the following steps work for integer values only

### Secret Sharing Using PySyft

We can share using the simple` .share()` method!

In [32]:
x = x.share(bob, alice, secure_worker) # implemets additive secrete shares
x # it's a pointer to three secrete shares

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:66463401384 -> bob:3885699908]
	-> (Wrapper)>[PointerTensor | me:27762259396 -> alice:92684798676]
	-> (Wrapper)>[PointerTensor | me:8418055714 -> secure_worker:45184251414]
	*crypto provider: me*

In [33]:
bob._objects # take a look at one of the shares.. Large random numbers --encrypted

{3885699908: tensor([3767212272783135879, 2599151911856180714, 2399438766361551658,
         2599575786773830802, 2167248781319205769]),
 28149015718: tensor([[0., 0.],
         [0., 1.]], requires_grad=True),
 74447433113: tensor([[0.],
         [0.]], requires_grad=True),
 89980499913: tensor([[ 0.0356],
         [-0.0119]], grad_fn=<AddmmBackward>)}

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [34]:
y = x + x
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:11967155743 -> bob:48292791669]
	-> (Wrapper)>[PointerTensor | me:59483504045 -> alice:80468040942]
	-> (Wrapper)>[PointerTensor | me:51844387304 -> secure_worker:91628338950]
	*crypto provider: me*

In [35]:
bob._objects # bob has another tensor.. why?

{3885699908: tensor([3767212272783135879, 2599151911856180714, 2399438766361551658,
         2599575786773830802, 2167248781319205769]),
 28149015718: tensor([[0., 0.],
         [0., 1.]], requires_grad=True),
 48292791669: tensor([7534424545566271758, 5198303823712361428, 4798877532723103316,
         5199151573547661604, 4334497562638411538]),
 74447433113: tensor([[0.],
         [0.]], requires_grad=True),
 89980499913: tensor([[ 0.0356],
         [-0.0119]], grad_fn=<AddmmBackward>)}

In [36]:
y.get() # notice that we have doubled the values on each worker!

tensor([ 2,  4,  6,  8, 10])


### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using `.fix_precision()`
This is important when doing FL since the values you are sharing are decimal numbers.

**Q: what values are we sharing? what are the shared secretes?**

In [0]:
x = th.tensor([0.1, 0.2, 0.3, 0.4]) # a decimal tensor!

In [38]:
x

tensor([0.1000, 0.2000, 0.3000, 0.4000])

In [39]:
x = x.fix_prec() # call this function to encode the decimal tensor using fixed precision
x # notice that this is a tensor chain <-------

(Wrapper)>FixedPrecisionTensor>tensor([100, 200, 300, 400])

In [40]:
print(f'Type of x {type(x)}')
print(f'Type of x.child {type(x.child)}')
print(f'Type of x.child.child {type(x.child.child)}') # <-- this is how you get the data from a fixed precision tensor

Type of x <class 'syft.frameworks.torch.tensors.interpreters.native.Tensor'>
Type of x.child <class 'syft.frameworks.torch.tensors.interpreters.precision.FixedPrecisionTensor'>
Type of x.child.child <class 'syft.frameworks.torch.tensors.interpreters.native.Tensor'>


In [41]:
# to decode the previous tensor
print(x.float_prec())

tensor([0.1000, 0.2000, 0.3000, 0.4000])


In [42]:
y = x + x
y

(Wrapper)>FixedPrecisionTensor>tensor([200, 400, 600, 800])

In [43]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000, 0.8000])

### Shared Fixed Precision

And of course, we can combine the two!

In [0]:
x = th.tensor([0.1, 0.2, 0.3])

In [45]:
x = x.fix_prec().share(bob, alice)
x

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:13788272265 -> bob:72214436720]
	-> (Wrapper)>[PointerTensor | me:9259627550 -> alice:73342859214]
	*crypto provider: me*

In [46]:
bob._objects

{3885699908: tensor([3767212272783135879, 2599151911856180714, 2399438766361551658,
         2599575786773830802, 2167248781319205769]),
 28149015718: tensor([[0., 0.],
         [0., 1.]], requires_grad=True),
 72214436720: tensor([4074315900972238991, 2170302416825340283, 4414860727311704267]),
 74447433113: tensor([[0.],
         [0.]], requires_grad=True),
 89980499913: tensor([[ 0.0356],
         [-0.0119]], grad_fn=<AddmmBackward>)}

In [0]:
y = x + x

In [48]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

# Final Project: Federated Learning with Encrypted Gradient Aggregation

Reuse your project from the Secuered Agggregator above to train the same model using the FL appraoch, but this time use the Additive Sharing Encryption so that the participating members have access to their own models only. There is no secure aggregator in this senario. 
Include four members in this project (Alice, Bob, Ted, and Carol). 

Hint: we will *share* the model with the shareholders, do addition on the encrypted models, and then aggregate the final model locally. 

In [68]:
import syft as sy
import torch as th
from torch import nn, optim

hook = sy.TorchHook(th)

W0711 23:51:03.892950 140657581631360 hook.py:98] Torch was already hooked... skipping hooking process


In [0]:
# create cells as needed. Separate your solution into logical cells.

# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
ted = sy.VirtualWorker(hook, id="ted")
carol = sy.VirtualWorker(hook, id="carol")
crypto_provider = sy.VirtualWorker(id="crypto_provider", hook=hook)

In [70]:
bob.clear_objects()
alice.clear_objects()
ted.clear_objects()
carol.clear_objects()


<VirtualWorker id:carol #objects:0>

In [0]:
# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.],[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.],[0],[0],[1],[1.]], requires_grad=True)

In [0]:
data = data.fix_precision().share(bob, alice, ted, carol, crypto_provider=crypto_provider,  requires_grad=True)
target = target.fix_precision().share(bob, alice, ted, carol, crypto_provider=crypto_provider,  requires_grad=True)

In [0]:
# Iniitalize A Toy Model
model = nn.Linear(2,1)

In [0]:
model = model.fix_precision().share(bob, alice, ted, carol, crypto_provider=crypto_provider,  requires_grad=True)
opt = optim.SGD(params=model.parameters(), lr=0.1).fix_precision()

In [76]:
for i in range(5):

    opt.zero_grad()
    pred = model(data)
    loss = ((pred - target)**2).sum()
    

    opt.step()
    print(loss.get().float_precision())

tensor([1.2890e+13])
tensor([8.6685e+12])
tensor([-4.2531e+13])
tensor([-1.1393e+13])
tensor([-4.0045e+13])


In [77]:
model = model.get()
print(model.float_precision().weight)

Parameter containing:
tensor([[ 0.4900, -0.5520]], requires_grad=True)
